In [1]:
!pip install -U spacy

In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

# Initialize

In [18]:
# have added norwegian date format in this english text
forward_lines_as_spacy_ner_format = [{'text': '3. Fire, machinery and electric installations\n',
  'ents': [],
  'title': 0},
 {'text': '\n', 'ents': [], 'title': 1},
 {'text': 'Section 7. Scope of application of this chapter\n',
  'ents': [],
  'title': 2},
 {'text': '(1) This chapter applies to vessels constructed after 1. januar 1992.\n',
  'ents': [],
  'title': 3},
 {'text': '(2) Vessels constructed before 1. januar 1992 shall be constructed and equipped in accordance with the rules applicable at the time of construction of the vessel, unless otherwise provided by the individual provisions of this chapter.\n',
  'ents': [],
  'title': 4},
 {'text': '(3) When a vessel is substantially altered or modified, the vessel shall comply with the requirements for a new vessel.\n',
  'ents': [],
  'title': 5},
 {'text': '\n', 'ents': [], 'title': 6},
 {'text': '0\tAmended by Regulation of 1. januar 1992 No.. 445 (in force on 1. januar 1992).\n',
  'ents': [],
  'title': 7},
 {'text': '\n', 'ents': [], 'title': 8},
 {'text': '\n', 'ents': [], 'title': 9},
 {'text': '\n', 'ents': [], 'title': 10},
 {'text': 'Section 8. Fire safety\n', 'ents': [], 'title': 11},
 {'text': '(1) Vessels constructed after 1. januar 1992 shall have structural fire protection in accordance with chapter C14 of Nordic Boat Standard (1990), unless otherwise provided by the individual provisions of this chapter.\n',
  'ents': [],
  'title': 12},
 {'text': '(2) Vessels constructed before 1. januar 1992 shall have structural fire protection in accordance with the rules applicable at the time of construction of the vessel.\n',
  'ents': [],
  'title': 13},
 {'text': '(3) Irrespective of the year of construction of the vessel, gasfired installations shall satisfy the requirements of Regulations of 1. januar 1992 No. 1580 on safety precautions for gasfired installations, etc. operating on propane or other liquefied hydrocarbon gases used on board vessels.\n',
  'ents': [],
  'title': 14},
 {'text': '\n', 'ents': [], 'title': 15},
 {'text': 'Section 9. Oil-fired stoves\n', 'ents': [], 'title': 16},
 {'text': '(1) Stoves with electric power supply shall have an oil supply which shuts off automatically if:\n',
  'ents': [],
  'title': 17}]

# Code

In [22]:
import spacy
#from spacy.lang.en import English
from spacy.lang.nb import Norwegian
from spacy.matcher import Matcher

# Blank model Norwegian().
# Does not have a POS tagger.
nlp = Norwegian() 

text = "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg."

doc = nlp(text)

matcher = Matcher(nlp.vocab)

date_pattern = [{"TEXT": {"REGEX": "[0-9]+[,]+[0-9]|[0-9]"}},
{"LOWER": {"IN": ["og"]}},
{"TEXT": {"REGEX": "[0-9]+[,]+[0-9]|[0-9]"}},
{"LOWER": {"IN": ["meter"]}}]

matcher.add("LENGTH-OVERALL", None, date_pattern)

result = []

for match_id, token_start, token_end in matcher(doc):

    # convert token_span to char_span.
    # char_span is needed to display correctly withdisplacy.render().
    span = doc[token_start:token_end]
    span_char_start = span[0].idx
    span_char_end = span[-1].idx + len(span[-1].text)

    match_id_as_string = nlp.vocab.strings[match_id]

    identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
    result.append(identified_entity)

#result

# display result with spacy styling
display_this_result = {'text': text, 'ents': sorted(result, key=lambda x: x['start']), 'title': 0}
options = {"colors": {"LENGTH-OVERALL": "yellow"}}
from spacy import displacy
displacy.render(display_this_result, style='ent', manual=True, options=options)

# Functions run from .py file

In [17]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_date_no

from spacy_matching_rule_identify_date_no import identify_date_in_text

importlib.reload(spacy_matching_rule_identify_date_no)

################
################

identify_date_in_text("(4) Fartøy med største lengde 6 til 15 meter bygget etter 1. januar 2002 med elektrisk anlegg med spenning over 50 V skal tilfredsstille kravene fastsatt i forskrift 4. desember 2001 nr. 1450 om maritime elektriske anlegg utgitt av Direktoratet for samfunnssikkerhet og beredskap.")

[{'start': 58, 'end': 72, 'label': 'DATE'},
 {'start': 166, 'end': 182, 'label': 'DATE'}]

In [19]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_date_no

from spacy_matching_rule_identify_date_no import identify_date_in_text

importlib.reload(spacy_matching_rule_identify_date_no)

################
################

for line in forward_lines_as_spacy_ner_format:

    text = line['text']

    if text != "\n":
        result_in_a_list = identify_date_in_text(text)

        if result_in_a_list:
            print(result_in_a_list)

[{'start': 54, 'end': 68, 'label': 'DATE'}]
[{'start': 31, 'end': 45, 'label': 'DATE'}]
[{'start': 27, 'end': 41, 'label': 'DATE'}, {'start': 64, 'end': 78, 'label': 'DATE'}]
[{'start': 30, 'end': 44, 'label': 'DATE'}]
[{'start': 31, 'end': 45, 'label': 'DATE'}]
[{'start': 132, 'end': 146, 'label': 'DATE'}]


# Merge spacy result with spacy ner formated list of text

In [20]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_date_no

from spacy_matching_rule_identify_date_no import merge_spacy_entity_results_to_spacy_ner_format

importlib.reload(spacy_matching_rule_identify_date_no)

################
################

spacy_ner_formated_text_line = {'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n',
  'ents': [],
  'title': 53}

spacy_ner_entities_to_be_merged_in_as_a_list = [{'start': 16, 'end': 19, 'label': 'DATE'}, {'start': 6, 'end': 9, 'label': 'DATE'}]

print(merge_spacy_entity_results_to_spacy_ner_format(spacy_ner_formated_text_line,spacy_ner_entities_to_be_merged_in_as_a_list))

spacy_ner_formated_text_line = {'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n',
  'ents': [{'start': 16, 'end': 19, 'label': 'DATE'}, {'start': 6, 'end': 9, 'label': 'DATE'}],
  'title': 53}

print("")

print(merge_spacy_entity_results_to_spacy_ner_format(spacy_ner_formated_text_line,spacy_ner_entities_to_be_merged_in_as_a_list))

{'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n', 'ents': [{'start': 6, 'end': 9, 'label': 'DATE'}, {'start': 16, 'end': 19, 'label': 'DATE'}], 'title': 53}

{'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material 

# Test the merger function on the data abowe

In [21]:
result = []

for spacy_line in forward_lines_as_spacy_ner_format:

    print(str(spacy_line['title']))

    if spacy_line['text'] != "\n" and len(spacy_line['text']) > 0:

        identify_date_in_text_result_in_a_list = identify_date_in_text(spacy_line['text'])

        # this result might be empty.

        if identify_date_in_text_result_in_a_list:

            # we have results to merge in.
            # merge in the result using this function.
            new_spacy_line = merge_spacy_entity_results_to_spacy_ner_format(spacy_line,identify_date_in_text_result_in_a_list)

            print(str(spacy_line['title']) + " ok if")
            result.append(new_spacy_line)

        else:
            print(str(spacy_line['title']) + " ok else")
            result.append(spacy_line)
    else:
        print(str(spacy_line['title']) + " ok else")
        result.append(spacy_line)

result

0
0 ok else
1
1 ok else
2
2 ok else
3
3 ok if
4
4 ok if
5
5 ok else
6
6 ok else
7
7 ok if
8
8 ok else
9
9 ok else
10
10 ok else
11
11 ok else
12
12 ok if
13
13 ok if
14
14 ok if
15
15 ok else
16
16 ok else
17
17 ok else


[{'text': '3. Fire, machinery and electric installations\n',
  'ents': [],
  'title': 0},
 {'text': '\n', 'ents': [], 'title': 1},
 {'text': 'Section 7. Scope of application of this chapter\n',
  'ents': [],
  'title': 2},
 {'text': '(1) This chapter applies to vessels constructed after 1. januar 1992.\n',
  'ents': [{'start': 54, 'end': 68, 'label': 'DATE'}],
  'title': 3},
 {'text': '(2) Vessels constructed before 1. januar 1992 shall be constructed and equipped in accordance with the rules applicable at the time of construction of the vessel, unless otherwise provided by the individual provisions of this chapter.\n',
  'ents': [{'start': 31, 'end': 45, 'label': 'DATE'}],
  'title': 4},
 {'text': '(3) When a vessel is substantially altered or modified, the vessel shall comply with the requirements for a new vessel.\n',
  'ents': [],
  'title': 5},
 {'text': '\n', 'ents': [], 'title': 6},
 {'text': '0\tAmended by Regulation of 1. januar 1992 No.. 445 (in force on 1. januar 1992).\n',


# identify_date_in_spacy_lines

In [22]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_date_no

from spacy_matching_rule_identify_date_no import identify_date_in_spacy_lines

importlib.reload(spacy_matching_rule_identify_date_no)

################
################

result = identify_date_in_spacy_lines(forward_lines_as_spacy_ner_format)
#print(result)

options = {"colors": {"DATE": "yellow"}}

# display result with spacy styling
from spacy import displacy
displacy.render(result, style='ent', manual=True, options=options)